# Requirement

In [1]:
## jalankan ini dahulu sebelum running
%pip install -qq -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# A. Case: Detik

In [24]:
## global
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup #scraping web statis
import time
from tqdm import tqdm #info progress
from urllib.parse import quote_plus, quote #parsing string "spasi" menjadi "%20" dan "+"

In [25]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://www.detik.com'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}
response = s.get(url, headers=headers, timeout=60)
print(response.status_code)

200


### A1. Crawling URLs dengan memasukkan keyword/query
```https://www.detik.com/search/searchall?query=makan%20bergizi%20gratis&page=5&result_type=relevansi``` -> pakai modulo 20

```https://www.detik.com/search/searchall?query=makan+bergizi+gratis&page=5&result_type=relevansi``` -> pakai +


```https://www.detik.com/search/searchall?query={kata_kunci}&page={halaman}&result_type=relevansi```

In [26]:
url = f"https://www.detik.com/search/searchall?query=makan%20bergizi&page=1&result_type=relevansi"
response = requests.get(url, headers=headers, timeout=60)
soup = BeautifulSoup(response.content, 'html.parser')
time.sleep(1)

In [27]:
## fungsi untuk scraping artikel berdasarkan keyword yang diinputkan
def scrape_detik_byquery(kata_kunci, halaman) -> pd.DataFrame:
    # Parameter input
    keyword = kata_kunci
    max_pages = halaman
    results = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    }

    for page in tqdm(range(1, max_pages + 1)):
        encoded_query = quote(keyword)
        url = f"https://www.detik.com/search/searchall?query={encoded_query}&page={page}&result_type=relevansi"
        try:
            response = requests.get(url, headers=headers, timeout=60)
            soup = BeautifulSoup(response.content, 'html.parser')
            time.sleep(1)

            beritas = soup.find_all('div', class_="media__text")
            for berita in beritas:
                try:
                    a = berita.find('a')
                    d = berita.find('div', class_="media__desc")
                    t = berita.find('div', class_="media__date")
                    k = berita.find('h2', class_="media__subtitle")

                    results.append({
                        "judul": a.text.strip() if a else np.nan,
                        "link": a.get('href') if a else np.nan,
                        "desc": d.text.strip() if d else np.nan,
                        "tanggal": t.text.strip() if t else np.nan,
                        "kategori": k.text.strip() if k else np.nan,
                        "keyword": keyword
                    })
                except Exception as e:
                    print("Skip 1 berita:", e)
        except Exception as e:
            print(f"Skip page {page}:", e)

    df = pd.DataFrame(results)
    print("Selesai. Total berita:", len(df))
    return df

In [28]:
df = scrape_detik_byquery("makan bergizi gratis", 3)
df.info()

100%|██████████| 3/3 [00:08<00:00,  2.76s/it]

Selesai. Total berita: 36
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     36 non-null     object
 1   link      36 non-null     object
 2   desc      30 non-null     object
 3   tanggal   36 non-null     object
 4   kategori  36 non-null     object
 5   keyword   36 non-null     object
dtypes: object(6)
memory usage: 1.8+ KB


In [29]:
df.tail(4)

,judul,link,desc,tanggal,kategori,keyword
32,"Pangan Lokal, Serangga, dan Makan Bergizi Gratis",https://news.detik.com/kolom/d-7782510/pangan-...,Program Makan Bergizi Gratis mengajarkan penti...,"Selasa, 18 Feb 2025 16:15 WIB",detikNews,makan bergizi gratis
33,Makan Bergizi Gratis Tetap Berjalan saat Ramadan,https://www.detik.com/sumut/berita/d-7753148/m...,Badan Gizi Nasional memastikan program Makan B...,"Selasa, 28 Jan 2025 15:30 WIB",detikSumut,makan bergizi gratis
34,Anggaran Makan Bergizi Gratis Mau Ditambah,https://www.detik.com/sumut/berita/d-7731493/a...,Rencana penambahan anggaran program Makan Berg...,"Selasa, 14 Jan 2025 13:20 WIB",detikSumut,makan bergizi gratis
35,Makan Bergizi Gratis dan Kapasitas Keuangan Ne...,https://news.detik.com/kolom/d-7771364/makan-b...,"Pada akhirnya, program MBG juga diharapkan dap...","Senin, 10 Feb 2025 15:30 WIB",detikNews,makan bergizi gratis


"Dedi Mulyadi kirim siswa nakal"

"barak militer siswa Jawa Barat"

"pelatihan militer siswa nakal"

"pendidikan karakter Dedi Mulyadi"

"kontroversi siswa ke militer"


In [30]:
## looping list keyword

# Daftar keyword yang ingin di-scrape
list_keyword = [
    "Dedi Mulyadi kirim siswa nakal",
    "barak militer siswa Jawa Barat",
    "pelatihan militer siswa nakal",
    "pendidikan karakter Dedi Mulyadi",
    "kontroversi siswa ke militer"
]

# Set jumlah halaman yang ingin di-scrape per keyword
halaman = 5

# List untuk menyimpan semua DataFrame
dfs = []

for keyword in list_keyword:
    print(f"Scraping untuk keyword: {keyword}")
    df_keyword = scrape_detik_byquery(keyword, halaman)
    dfs.append(df_keyword)

# Gabungkan semua hasil
df = pd.concat(dfs, ignore_index=True)

Scraping untuk keyword: Dedi Mulyadi kirim siswa nakal


100%|██████████| 5/5 [00:16<00:00,  3.29s/it]


Selesai. Total berita: 51
Scraping untuk keyword: barak militer siswa Jawa Barat


100%|██████████| 5/5 [00:14<00:00,  2.85s/it]


Selesai. Total berita: 50
Scraping untuk keyword: pelatihan militer siswa nakal


...

Selesai. Total berita: 50
Scraping untuk keyword: pendidikan karakter Dedi Mulyadi


100%|██████████| 5/5 [00:16<00:00,  3.20s/it]


Selesai. Total berita: 50
Scraping untuk keyword: kontroversi siswa ke militer


100%|██████████| 5/5 [00:13<00:00,  2.75s/it]

Selesai. Total berita: 50


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     251 non-null    object
 1   link      251 non-null    object
 2   desc      250 non-null    object
 3   tanggal   251 non-null    object
 4   kategori  251 non-null    object
 5   keyword   251 non-null    object
dtypes: object(6)
memory usage: 11.9+ KB


In [32]:
# Meremove duplicate berita
df_final = df.drop_duplicates(subset=['judul', 'link'], keep='first')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 0 to 250
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     155 non-null    object
 1   link      155 non-null    object
 2   desc      155 non-null    object
 3   tanggal   155 non-null    object
 4   kategori  155 non-null    object
 5   keyword   155 non-null    object
dtypes: object(6)
memory usage: 8.5+ KB


In [33]:
df_final.to_csv("./files/detik_links.csv", index=False, encoding='utf-8-sig')

### A2. Scraping satu artikel

In [34]:
## read data
df_read = pd.read_csv("./files/detik_links.csv", encoding='utf-8-sig')
df_read.head()

,judul,link,desc,tanggal,kategori,keyword
0,Video: Pramono Anung Ogah Tiru Dedi Mulyadi Ki...,https://20.detik.com/detikupdate/20250503-2505...,Gubernur DKI Jakarta Pramono Anung mengaku tak...,"Sabtu, 03 Mei 2025 21:37 WIB",detikNews,Dedi Mulyadi kirim siswa nakal
1,Zaini Shofari Minta Pembatasan Jam Malam Pelaj...,https://www.detik.com/jabar/berita/d-7934892/z...,Anggota DPRD Jabar Zaini Shofari dukung pembat...,19 jam yang lalu,detikJabar,Dedi Mulyadi kirim siswa nakal
2,Blak-Blakan Dedi Mulyadi soal Kirim Anak Nakal...,https://www.detik.com/jabar/berita/d-7895715/b...,Gubernur Dedi Mulyadi kirim siswa nakal di Jaw...,"Jumat, 02 Mei 2025 12:00 WIB",detikJabar,Dedi Mulyadi kirim siswa nakal
3,Kata Menko PMK Soal Kebijakan Anak Nakal Dikir...,https://www.detik.com/jatim/berita/d-7918739/k...,Menteri Koordinator Bidang Pembangunan Manusia...,"Sabtu, 17 Mei 2025 17:00 WIB",detikJatim,Dedi Mulyadi kirim siswa nakal
4,Kata Raffi Ahmad Saat Rafathar Ngadu ke KDM so...,https://hot.detik.com/celeb/d-7914445/kata-raf...,Raffi Ahmad menjelaskan tentang Nagita Slavina...,"Rabu, 14 Mei 2025 22:01 WIB",detikHot,Dedi Mulyadi kirim siswa nakal


In [35]:
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     155 non-null    object
 1   link      155 non-null    object
 2   desc      155 non-null    object
 3   tanggal   155 non-null    object
 4   kategori  155 non-null    object
 5   keyword   155 non-null    object
dtypes: object(6)
memory usage: 7.4+ KB


In [36]:
df['keyword'].unique()

array(['Dedi Mulyadi kirim siswa nakal', 'barak militer siswa Jawa Barat',
       'pelatihan militer siswa nakal',
       'pendidikan karakter Dedi Mulyadi', 'kontroversi siswa ke militer'],
      dtype=object)

In [44]:
## fungsi melakukan scraping satu artikel
def scrape_detik_satu(url: str) -> pd.DataFrame:
    response = requests.get(url, headers=headers, timeout=60)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)
        try:
            # Pisahkan jadi hari, tanggal, dan jam
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    # Ambil kategori
    kat_div = soup.find('div', class_='page__breadcrumb')
    kategori = np.nan
    sub_kategori = np.nan
    if kat_div:
        kategori = kat_div.find('a').get_text(strip=True)
        a_tag = kat_div.find('a', attrs={'dtr-ttl': True})
        if a_tag:
            sub_kategori = a_tag.get('dtr-ttl')

    #Ambil Isi
    final_paragraphs = soup.find_all('p')
    final_isi = '\n\n'.join(p.get_text(strip=True) for p in final_paragraphs)
    
    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": final_isi,
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        # "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "kategori": kategori,
        "sub_kategori": sub_kategori,
        "link": url
    }

    df = pd.DataFrame([hasil])
    return df

In [45]:
df_read["link"][1]

'https://www.detik.com/jabar/berita/d-7934892/zaini-shofari-minta-pembatasan-jam-malam-pelajar-libatkan-polisi'

In [46]:
scrape_detik_satu(df_read["link"][1])

,judul,isi,hari,tanggal,jam,kategori,sub_kategori,link
0,Zaini Shofari Minta Pembatasan Jam Malam Pelaj...,"Anggota Komisi V DPRD Jawa Barat, Zaini Shofar...",Selasa,27 Mei 2025,16:00 WIB,detikJabar,Berita,https://www.detik.com/jabar/berita/d-7934892/z...


### A3. Scraping artikel dari URLs yang telah diperoleh


In [47]:
def scrape_detik_dari_csv(path_csv: str) -> pd.DataFrame:
    # Membaca CSV hasil scraping link
    df_links = pd.read_csv(path_csv)

    # Memastikan kolom 'link' ada
    if 'link' not in df_links.columns:
        raise ValueError("CSV tidak mengandung kolom 'link'.")

    hasil_semua = []
    for i,row in tqdm(df_links.iterrows(), total=len(df_links)): #tuple (i,series)
        url = row['link']
        df_artikel = scrape_detik_satu(url) #memanggil dan menjalankan fungsi scrape_detik satu artikel
        keyword = row['keyword']
        if df_artikel is not None:
            df_artikel['keyword'] = keyword
            hasil_semua.append(df_artikel)

    # Gabungkan semua DataFrame
    if hasil_semua:
        df_final = pd.concat(hasil_semua, ignore_index=True)
        df_final.to_csv('./files/detik_semua_artikel_query.csv', index=False, encoding='utf-8-sig') #disesuaikan dengan path teman2
        print("Selesai menyimpan semua artikel.")
        return df_final
    else:
        print("Tidak ada artikel yang berhasil di-scrape.")
        return pd.DataFrame()

In [48]:
df = scrape_detik_dari_csv('./files/detik_links.csv')
df.info()

100%|██████████| 155/155 [05:40<00:00,  2.20s/it]

Selesai menyimpan semua artikel.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   judul         152 non-null    object
 1   isi           155 non-null    object
 2   hari          146 non-null    object
 3   tanggal       146 non-null    object
 4   jam           146 non-null    object
 5   kategori      129 non-null    object
 6   sub_kategori  129 non-null    object
 7   link          155 non-null    object
 8   keyword       155 non-null    object
dtypes: object(9)
memory usage: 11.0+ KB


In [49]:
df.head(3)

,judul,isi,hari,tanggal,jam,kategori,sub_kategori,link,keyword
0,Video: Pramono Anung Ogah Tiru Dedi Mulyadi Ki...,Gubernur DKI Jakarta Pramono Anung mengaku tak...,9,"839 Views | Sabtu, 03 Mei 2025",20:51 WIB,NaN,NaN,https://20.detik.com/detikupdate/20250503-2505...,Dedi Mulyadi kirim siswa nakal
1,Zaini Shofari Minta Pembatasan Jam Malam Pelaj...,"Anggota Komisi V DPRD Jawa Barat, Zaini Shofar...",Selasa,27 Mei 2025,16:00 WIB,detikJabar,Berita,https://www.detik.com/jabar/berita/d-7934892/z...,Dedi Mulyadi kirim siswa nakal
2,Blak-Blakan Dedi Mulyadi soal Kirim Anak Nakal...,Siswa sekolah yang dianggap nakal di Jawa Bara...,Jumat,02 Mei 2025,12:00 WIB,detikJabar,Berita,https://www.detik.com/jabar/berita/d-7895715/b...,Dedi Mulyadi kirim siswa nakal


In [50]:
df_clean = df.dropna()

In [51]:
df_clean.to_csv('./files/detik_semua_artikel_query_clean.csv', index=False, encoding='utf-8-sig')
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129 entries, 1 to 153
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   judul         129 non-null    object
 1   isi           129 non-null    object
 2   hari          129 non-null    object
 3   tanggal       129 non-null    object
 4   jam           129 non-null    object
 5   kategori      129 non-null    object
 6   sub_kategori  129 non-null    object
 7   link          129 non-null    object
 8   keyword       129 non-null    object
dtypes: object(9)
memory usage: 10.1+ KB


### A4. Detik.com menyediakan sitemap xml

In [ ]:
# Ambil sitemap (allowed dari detik.com/robots.txt)
url_sitemap = 'https://www.detik.com/sitemap.xml'
response = requests.get(url_sitemap)
soup = BeautifulSoup(response.content, 'xml')  # parsing sebagai XML

# Ambil semua URL sitemap yang disediakan
sitemap_urls = [loc.text for loc in soup.find_all('loc')]
print("Contoh:", sitemap_urls[0])
print("Total sitemap url:", len(sitemap_urls))

In [ ]:
## cek satu sitemap ada berapa artikel
sitemap_berita = sitemap_urls[0] 
resp = requests.get(sitemap_berita)
soup2 = BeautifulSoup(resp.content, 'xml')
artikel_urls = [loc.text.strip() for loc in soup2.find_all('loc')]
print("Total artikel:", len(artikel_urls))

In [ ]:
artikel_urls[0]

In [ ]:
berita = []
for url in artikel_urls[:5]:  # batasi dulu misalnya 5
    try:
        hasil = scrape_detik_satu(url)
        berita.append(hasil)
    except Exception as e:
        print("Gagal:", url, e)

df = pd.concat(berita, ignore_index=True)
df

### A5. Scraping seluruh isi sitemap

In [ ]:
#fungsi ambil url sitemap dari sitemap utama
def get_sitemap_urls(master_sitemap_url: str):
    response = requests.get(master_sitemap_url, headers=headers, timeout=60)
    soup = BeautifulSoup(response.content, 'xml')
    sitemap_urls = [loc.get_text().strip() for loc in soup.find_all('loc')]
    return sitemap_urls

#fungsi ambil url article
def get_article_urls(sitemap_url: str):
    response = requests.get(sitemap_url, headers=headers, timeout=60)
    soup = BeautifulSoup(response.content, 'xml')
    article_urls = [loc.get_text().strip() for loc in soup.find_all('loc')]
    return article_urls

def scrape_detik(url: str):
    response = requests.get(url, headers=headers, timeout=60)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal utuh
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)
        try:
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": "\n\n".join(p.get_text(strip=True) for p in soup.find_all('p')),
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "link": url
    }

    return hasil

In [ ]:
def scrape_all_from_sitemap(master_sitemap_url: str, max_sitemap=None, max_articles=None, output_csv: str="detik_semua_artikel_sitemap.csv"):
    """
    Robot scraping semua artikel dari sitemap utama Detik.

    Args:
        master_sitemap_url (str): URL sitemap utama.
        max_sitemap (int, optional): Batasi jumlah sitemap yang di-scrape. None artinya semua.
        max_articles (int, optional): Batasi jumlah artikel per sitemap. None artinya semua.
        output_csv (str): Nama file output CSV.

    Returns:
        pd.DataFrame: DataFrame berisi hasil scraping artikel.
    """
    all_articles = []

    print("Mengambil daftar sitemap...")
    sitemap_urls = get_sitemap_urls(master_sitemap_url)

    if max_sitemap:
        sitemap_urls = sitemap_urls[:max_sitemap]

    for sitemap in tqdm(sitemap_urls, desc="Sitemap"):
        try:
            article_urls = get_article_urls(sitemap)
            if max_articles:
                article_urls = article_urls[:max_articles]

            for url in tqdm(article_urls, desc="Artikel", leave=False):
                try:
                    article = scrape_detik(url)
                    all_articles.append(article)
                    time.sleep(1)  # Hindari ban
                except Exception as e:
                    print("Gagal scraping artikel:", url, e)

        except Exception as e:
            print("Gagal akses sitemap:", sitemap, e)

    # Simpan ke CSV
    df = pd.DataFrame(all_articles)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"Selesai. Total artikel: {len(df)}. Hasil disimpan di {output_csv}")
    return df

In [ ]:
df = scrape_all_from_sitemap('https://www.detik.com/sitemap.xml', max_sitemap=2, max_articles=10)
df.head()

# B. Case: Tempo

In [20]:
## global
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup #scraping web statis
import time
from tqdm import tqdm #info progress

## selenium, scraping web dinamis
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [21]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://www.tempo.co/search?q=makan+bergizi+gratis&page=1'
response = s.get(url)
print(response.status_code)

200


### B1. Crawling URLs

In [22]:
##Fungsi melakukan scraping URL Tempo, masukkan string keyword dan max halaman
def scrape_tempo_search_selenium(kata_kunci: str, halaman: int) -> pd.DataFrame: 
    """
    Robot crawling url yang diinginkan berdasarkan kata kunci yang user input.

    Args:
        kata_kunci (str): Query yang ingin dimasukkan.
        halaman (int): Batasi jumlah halaman yang di-scrape.

    Returns:
        pd.DataFrame: DataFrame berisi hasil crawling url.
    """
    # Set User-Agent
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    opts = Options()
    opts.add_argument(f"user-agent={user_agent}")
    opts.add_argument("--headless")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
        
    # Parameter input
    keyword = kata_kunci #contoh: "makan bergizi gratis"
    max_pages = halaman #contoh: 2
    results = []

    # Loop halaman
    for page in tqdm(range(1, max_pages + 1)):
        # print(f"Scraping page {page}...")

        # Format URL pencarian
        encoded_query = quote_plus(keyword)
        url = f"https://www.tempo.co/search?q={encoded_query}&page={page}"
        
        driver.get(url)
        time.sleep(10)

        try:
            container = driver.find_element("css selector", "div.flex.flex-col.divide-y.divide-neutral-500")
            beritas = container.find_elements("css selector", "figure.flex.flex-row.gap-3.py-4.container.lg\\:mx-0.lg\\:px-0")
            for berita in beritas:
                try:
                    a = berita.find_element("tag name", "a")
                    p = berita.find_element("tag name", "p")
                    results.append({
                        "judul": p.text,
                        "link": a.get_attribute("href"),
                        "keyword": keyword
                    })
                except Exception as e:
                    print("Skip 1 berita:", e)

        except Exception as e:
            print("Skip page:", e)

    driver.quit()

    # Simpan ke DataFrame
    df = pd.DataFrame(results)
    print("Selesai. Total berita:", len(df))
    return df

In [23]:
#max pages yang bisa diakses publik hanya 100 pages untuk tiap keyword (pengecekan manual)
scrape_tempo_search_selenium("makan bergizi gratis", 2)

100%|██████████| 2/2 [01:08<00:00, 34.45s/it]


Selesai. Total berita: 20


,judul,link,keyword
0,Sinyal Darurat Penerimaan Pajak,https://www.tempo.co/kolom/rasio-pajak-penerim...,makan bergizi gratis
1,Seknas Fitra: Makan Bergizi Gratis Belum Dongk...,https://www.tempo.co/ekonomi/seknas-fitra-maka...,makan bergizi gratis
2,"Hingga 21 Mei, Anggaran Makan Bergizi Gratis S...",https://www.tempo.co/ekonomi/hingga-21-mei-ang...,makan bergizi gratis
3,Korban Keracunan MBG Lebih Besar dari Klaim Pr...,https://www.tempo.co/politik/korban-keracunan-...,makan bergizi gratis
4,"Sejumlah Titik Kritis dalam Program MBG, Menur...",https://www.tempo.co/politik/sejumlah-titik-kr...,makan bergizi gratis
5,Bos BGN Membantah Raffi Ahmad Kelola 300 Dapur...,https://www.tempo.co/ekonomi/bos-bgn-membantah...,makan bergizi gratis
6,Sekjen Kemendagri Instruksikan Percepatan Laha...,https://www.tempo.co/info-tempo/sekjen-kemenda...,makan bergizi gratis
7,Badan Gizi Nasional jadi Instansi dengan Pagu ...,https://www.tempo.co/ekonomi/badan-gizi-nasion...,makan bergizi gratis
8,BGN Tambah 294 SPPG untuk Makan Bergizi Gratis...,https://www.tempo.co/politik/bgn-tambah-294-sp...,makan bergizi gratis
9,REI: Program 3 Juta Rumah Dukung Efektivitas M...,https://www.tempo.co/ekonomi/rei-program-3-jut...,makan bergizi gratis


In [ ]:
## looping list keyword

# Daftar keyword yang ingin di-scrape
list_keyword = [
    "makan bergizi gratis",
    "efisiensi anggaran",
    "CPNS 2025",
    "kemiskinan world bank"
]

# Set jumlah halaman yang ingin di-scrape per keyword
halaman = 2

# List untuk menyimpan semua DataFrame
dfs = []

for keyword in list_keyword:
    print(f"Scraping untuk keyword: {keyword}")
    df_keyword = scrape_tempo_search_selenium(keyword, halaman)
    dfs.append(df_keyword)

# Gabungkan semua hasil
df_final = pd.concat(dfs, ignore_index=True)

In [ ]:
df_final.to_csv("./files/tempo_links.csv", index=False, encoding='utf-8-sig')
df_final

### B2. Scraping satu artikel

In [ ]:
##Fungsi melakukan scraping data 1 halaman Tempo, masukkan string url
def scrape_tempo(url: str) -> pd.DataFrame: 
    ## Inisiasi dictionary hasil
    hasil = {}

    try:
        response = requests.get(url, headers=headers, timeout=60)
        soup = BeautifulSoup(response.text, 'html.parser')

        ##Judul
        judul = soup.find('h1', class_='text-[26px] font-bold leading-[122%] text-neutral-1200')
        hasil['judul'] = judul.get_text(strip=True) if judul else np.nan

        ## Sub judul
        sub_judul = soup.find('div', class_='font-roboserif leading-[156%] text-neutral-1100')
        hasil['sub_judul'] = sub_judul.get_text(strip=True) if sub_judul else np.nan

        ## Isi berita
        isi_paragraf = []
        isi_berita = soup.find_all('div', id='content-wrapper', class_='max-lg:container xl')

        for i in isi_berita:
            paragraf = i.find_all('p')
            for p in paragraf:
                teks = p.get_text(strip=True)
                if teks:  #menambahkan teks bila ada
                    isi_paragraf.append(teks)
        ringkasan = '\n\n'.join(isi_paragraf)
        hasil['isi'] = ringkasan if ringkasan else np.nan

        ## Tanggal & Jam publikasi
        tanggal_publikasi = soup.find('p', class_='text-neutral-900 text-sm')
        if tanggal_publikasi:
            waktu = tanggal_publikasi.get_text(strip=True)
            if '|' in waktu:
                tanggal, jam = [part.strip() for part in waktu.split('|')]
                hasil['tanggal'] = tanggal
                hasil['jam'] = jam
            else:
                hasil['tanggal'] = waktu
                hasil['jam'] = np.nan
        else:
            hasil['tanggal'] = np.nan
            hasil['jam'] = np.nan

        ## Kategori
        kategori = soup.find('span', class_='text-sm font-medium text-primary-main')
        hasil['kategori'] = kategori.get_text(strip=True) if kategori else np.nan

        ## Link
        hasil['link'] = url 

    except Exception as e:
        print(f"Terjadi kesalahan saat scraping: {e}")
        return None

    ## Kembalikan juga sebagai DataFrame
    df = pd.DataFrame([hasil])
    # print('selesai scraping')
    return df

In [ ]:
url = 'https://www.tempo.co/ekonomi/potensi-masalah-dari-rencana-pemerintah-ubah-lapas-jadi-perumahan-1533913'
df_hasil = scrape_tempo(url)
df_hasil

### B3. Scraping artikel dalam URLs

In [ ]:
##Fungsi melakukan scraping data dari hasil crawling URL Tempo, masukkan csv
def scrape_tempo_dari_csv(path_csv: str) -> pd.DataFrame:
    # Membaca CSV hasil scraping link
    df_links = pd.read_csv(path_csv)

    # Memastikan kolom 'link' ada
    if 'link' not in df_links.columns:
        raise ValueError("CSV tidak mengandung kolom 'link'.")

    hasil_semua = []
    for i, row in tqdm(df_links.iterrows(), total=len(df_links)):
        url = row['link']
        df_artikel = scrape_tempo(url) #memanggil dan menjalankan fungsi scrape_tempo satu artikel
        keyword = row['keyword']
        if df_artikel is not None:
            df_artikel['keyword'] = keyword
            hasil_semua.append(df_artikel)

    # Gabungkan semua DataFrame
    if hasil_semua:
        df_final = pd.concat(hasil_semua, ignore_index=True)
        df_final.to_csv('./files/tempo_semua_artikel.csv', index=False, encoding='utf-8-sig')
        print("Selesai menyimpan semua artikel.")
        return df_final
    else:
        print("Tidak ada artikel yang berhasil di-scrape.")
        return pd.DataFrame()

In [ ]:
df = scrape_tempo_dari_csv("./files/tempo_links.csv")
df.head(5)

# Next: Analisis Sentimen